In [1]:
import codebase.web_scrape_functions as wsf
import codebase.analysis_functions as af
from codebase import match_data
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import utils
from utils import logger
import os
import logging
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codebase.graphing_functions as gf
from codebase.settings import CAREERS

%load_ext autoreload
%autoreload 2
logger.handlers[1].setLevel(logging.INFO)
# logger.disabled = True
pd.get_option("display.max_columns")

20

In [2]:
KOHLI_ID = '253802'
ROOT_PLAYER_ID = '303669'
WILLIAMSON_PLAYER_ID = '277906'
SPD_SMITH_ID = '267192'

Lets do a deep dive into Kohli's drives and innings in general. First lets do a few basic questions, average length of Kohli's innings, scores, minutes, etc.

In [3]:
kohli_matches = wsf.get_player_match_list(KOHLI_ID)


2022-07-29 17:52:08,655 - INFO - Getting match list for player: 253802 Dates: None


In [4]:
kohli_innings = af.get_cricket_totals(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True)
kohli_innings_df = pd.DataFrame(kohli_innings)

2022-07-29 17:52:15,003 - INFO - Getting player controbutions for match 489226
2022-07-29 17:52:15,006 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\match_jsons\489226_match_jsons.json
2022-07-29 17:52:15,011 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\htmls\489226_html.html
2022-07-29 17:52:15,264 - INFO - Match ID 489226: Loading match comms
2022-07-29 17:52:15,267 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\full_comms\489226_full_comms.json
2022-07-29 17:52:15,314 - INFO - Getting player totals for match: 489226 Player: 253802
2022-07-29 17:52:15,316 - INFO - Pre-transforming match commentary for 489226
2022-07-29 17:52:15,347 - INFO - 489226: Processing text commentary fields
2022-07-29 17:52:15,735 - INFO - 489226: Processing bowler runs
2022-07-29 17:52:16,487 - INFO - Getting player controbutions for match 489227
2022-07-29 17:52:16,504 - INFO - Lo

In [5]:
kohli_innings_df.head()

,inning,runs,balls_faced,fours,six,dot_balls,not_out,how_out,date,team,opposition,ground,continent,match_id
0,1,4,10,1,0,9,False,caught,2011-06-20,6,4,200,Americas,489226
1,3,15,54,2,0,43,False,caught,2011-06-20,6,4,200,Americas,489226
2,1,0,2,0,0,2,False,caught,2011-06-28,6,4,199,Americas,489227
3,3,27,107,1,1,87,False,caught,2011-06-28,6,4,199,Americas,489227
4,2,30,53,2,0,35,False,caught,2011-07-06,6,4,629,Americas,489228


In [6]:
kohli_innings_df.describe()

,inning,runs,balls_faced,fours,six,dot_balls
count,173.000000,173.000000,173.000000,173.000000,173.000000,173.000000
mean,2.213873,46.670520,84.046243,5.260116,0.138728,58.421965
std,1.070445,53.477023,81.056527,6.143296,0.393778,53.339533
min,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,1.000000,10.000000,21.000000,1.000000,0.000000,15.000000
50%,2.000000,28.000000,60.000000,3.000000,0.000000,44.000000
75%,3.000000,58.000000,113.000000,8.000000,0.000000,81.000000
max,4.000000,254.000000,366.000000,33.000000,2.000000,241.000000


In [7]:
pd.cut(kohli_innings_df.balls_faced, [0,10,20,30,40,50,100,150,200, float("inf")]).value_counts()

(50.0, 100.0]     43
(0.0, 10.0]       24
(100.0, 150.0]    23
(20.0, 30.0]      20
(200.0, inf]      19
(10.0, 20.0]      18
(150.0, 200.0]    12
(30.0, 40.0]      10
(40.0, 50.0]       4
Name: balls_faced, dtype: int64

Ok now back to the cover drives, we will deep dive, how many cover drives does he play in each bin, and the average of those cover drives. We wanna see if there is a trend in when he plays the shot and the average. First lets get all commentary and then we can filter the innings based on the bins and match to commentary

## Get all commentary from match and turn it to list

In [8]:
kohli_comms = af.get_player_contributions(KOHLI_ID, kohli_matches, _type = 'bat', by_innings=True, is_object_id=True)

2022-07-29 17:53:55,285 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\match_jsons\489226_match_jsons.json
2022-07-29 17:53:55,288 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\htmls\489226_html.html
2022-07-29 17:53:55,539 - INFO - Match ID 489226: Loading match comms
2022-07-29 17:53:55,540 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\full_comms\489226_full_comms.json
2022-07-29 17:53:55,585 - INFO - Pre-transforming match commentary for 489226
2022-07-29 17:53:55,612 - INFO - 489226: Processing text commentary fields
2022-07-29 17:53:56,029 - INFO - 489226: Processing bowler runs
2022-07-29 17:53:56,046 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\match_jsons\489227_match_jsons.json
2022-07-29 17:53:56,050 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\htmls\489227_html.h

In [9]:
kohli_comms[0].commentTextItems.tolist()

['Carefully gets forward to a tossed up ball and defends',
 'Turns away from a shortish length and virat goes back to push it towards point',
 'Another watchful front foot block from virat',
 "And it wasn't long coming, virat brings out the whiplash extra cover drive off a fuller one but cannot beat the infield",
 'On the front foot and pushed away towards silly mid off',
 'Virat gets a gift and accepts it to score his first runs in tests, the attempted yorker gone wrong, way down the leg side and virat just helps it away with a nudge',
 'There arrives the short ball, but a touch outside off stump and virat calmly weaves out of its path',
 'Gets behind a short of a length ball and blocks it back to the bowler',
 'This one jags back in but virat is not committing forward at all and stays in the crease to block it towards short leg',
 "That is the end of virat's debut innings, he is gone nibbling at one outside off and baugh has gobbled up the appetiser before lunch, was some distance fr

We have all the innings commentary, no lets get just the commentTextItems and then we can search in those to see when the commentary is cover drives

In [10]:
commentary = []
for match in kohli_comms:
    commentary.append(match.commentTextItems.tolist())

## Searching for cover drives

In [11]:
cover_drives = []
for inning in commentary:
    cover_drives.append(af.search_for_keywords(inning, ['drive', 'cover', 'defending', 'defence', 'defends'], exclude_words=['run out', 'pull', 'flick'],return_matching=True, return_indices=True))

In [12]:
cover_drives[3][2]

[4,
 14,
 17,
 20,
 21,
 22,
 26,
 29,
 37,
 39,
 41,
 43,
 44,
 47,
 48,
 49,
 50,
 55,
 57,
 58,
 66,
 70,
 74,
 75,
 77,
 79,
 80,
 84,
 85,
 86,
 87,
 88,
 89,
 95,
 96,
 106]

In [13]:
cover_drives[3][1]

["Whoa, what was that? edwards pitches one up, just seeing if kohli is ready for it, and he wasn't. 88 mph, and it was at kohli's feet by the time he knew it it was there. he wasn't forward in time, and threw his bat at a hard-handed drive. it popped up in the leg side and nearly carried to bishoo at midwicket,",
 'Kohli leans out and drives into the off side with the spin.',
 'Nervy moments for kohli. survives another bouncer without much control. bouncer heading for the ribs, he hops back and tries to ride the bounce but ends up limply providing the handle of the bat as his last line of defence. it could have gone anywhere, but managed to stay just away from barath at short leg.',
 "Good ball, 88 mph and full, seeing if kohli is ready to get forward. he wasn't quick enough to change tack, but went for the drive, and ended it onto the pad.",
 'Tossed up, kohli drives through mid-off for a single.',
 "Kohli wisely takes a single and moves to the bowler's end. short ball outside off, he

Now that we have the deliveries that kohli played drives to, we can reference this back to the df and pick out all the rows where he played a cover drive. Then we can get the average off the cover drive and dismissals etc. Next we will need to do this for other shots and start to build a picture of how Kohli gets out.

In [14]:
cover_drive_df = pd.DataFrame()
for i,inning in enumerate(cover_drives):
    indices = inning[2]
    cover_drive_df = pd.concat([cover_drive_df, kohli_comms[i].iloc[indices]])

In [15]:
cover_drive_df[cover_drive_df['isWicket'] == True].count()

_uid                    38
id                      38
inningNumber            38
ballsActual              0
ballsUnique              0
oversUnique             38
oversActual             38
overNumber              38
ballNumber              38
totalRuns               38
batsmanRuns             38
isFour                  38
isSix                   38
isWicket                38
dismissalType           38
byes                    38
legbyes                 38
wides                   38
noballs                 38
timestamp               11
batsmanPlayerId         38
bowlerPlayerId          38
totalInningRuns         38
title                   38
dismissalText           38
commentPreTextItems     38
commentTextItems        38
commentPostTextItems    38
commentVideos           37
events                  38
over                     7
batsmanName             38
bowlerName              38
bowlerRuns              38
battingTeam             38
commentImages            5
dtype: int64

Only 36 dismissals with cover drive in them, would have thought there were way more. What are the other ways Kohli is getting out? Lets create a dictionary of words that are appearing in Kohli dismissals. First we need to get all the dismissals.

## Get all dismissals

In [16]:
kohli_comms_flat = pd.DataFrame()
for comms in kohli_comms:
    kohli_comms_flat = pd.concat([kohli_comms_flat, comms])

In [17]:
kohli_comms_flat[kohli_comms_flat.isWicket == True].count()

_uid                    165
id                      165
inningNumber            165
ballsActual               0
ballsUnique               0
oversUnique             165
oversActual             165
overNumber              165
ballNumber              165
totalRuns               165
batsmanRuns             165
isFour                  165
isSix                   165
isWicket                165
dismissalType           165
byes                    165
legbyes                 165
wides                   165
noballs                 165
timestamp                34
batsmanPlayerId         165
bowlerPlayerId          165
totalInningRuns         165
title                   165
dismissalText           165
commentPreTextItems     165
commentTextItems        165
commentPostTextItems    165
commentVideos           162
events                  165
over                     24
batsmanName             165
bowlerName              165
bowlerRuns              165
battingTeam             165
commentImages       

In [18]:
kohli_dismissals = kohli_comms_flat[(kohli_comms_flat.isWicket == True) & (kohli_comms_flat.batsmanPlayerId == int(af.get_player_map(match_data.MatchData(kohli_matches[0]), 'player_id', 'object_id')[int(KOHLI_ID)]))]

2022-07-29 17:55:05,940 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\match_jsons\489226_match_jsons.json
2022-07-29 17:55:05,944 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\htmls\489226_html.html
2022-07-29 17:55:06,258 - INFO - Match ID 489226: Loading match comms
2022-07-29 17:55:06,261 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\full_comms\489226_full_comms.json


In [19]:
dismissals_list = kohli_dismissals.commentTextItems.to_list()

In [20]:
dismissals_list

["That is the end of virat's debut innings, he is gone nibbling at one outside off and baugh has gobbled up the appetiser before lunch, was some distance from virat, not much need to play at it, but for once virat pressed forward, and got so close to the ball that he had to play at it, and ended up getting a healthy edge through to the keeper, a happy bunch of west indians will take lunch,",
 "Soft dismissal! short ball, aimed into the ribs, but headed down the leg side, kohli flirts with it, and west indies' appeal for a catch down the leg side is spontaneous. harper agrees, and a disappointed kohli goes back. did he show dissent? were there any grounds for dissent? we know not yet... can't tell at all from these replays. as an aside, these are the incidents where drs becomes ineffective if there is no hot spot. makes no sense to have drs without proper technology",
 'Rampaul has taken off, his team-mates rush after him, virat has been bounced out, nothing shot, nothing at all actuall

In [21]:
cover_drive_dismissals = cover_drive_df[cover_drive_df['isWicket'] == True].commentTextItems.to_list()

In [22]:
cover_drive_dismissals

["Kohli's day just got worse, a loose drive away from his body, it was short of a length and outside off, perhaps should have left it alone, he decides to drive on the up instead, though edwards has been swinging the ball away, duly gets the outside edge and its easy for second slip",
 "They won't need the new ball for virat, he is gone, all the batsmen have fallen today to the pitched up ball which moves away slightly, kohli is forward in defence, but the movement has done him, and the outside edge is snapped up easily by haddin",
 "The end of kohli, he was looking so good this session, can't blame any demons in the pitch for that dismissal, a length ball that kohli drives uppishly, warner was waiting at point for that, some exuberant celebrations from siddle, kohli should be kicking himself to be dismissed in this manner after taking his time to settle in, and surviving a testing early phase",
 "Gone! panesar breaks through again. loose from kohli. hint of width outside off, his eyes

In [23]:
import nltk
from nltk.corpus import stopwords
exclude_words = set(stopwords.words('english'))
exclude_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [24]:
import string

vocabulary = {}

for dismissal in dismissals_list:
    dismissal = dismissal.translate(str.maketrans('', '', string.punctuation))
    wordlist = dismissal.split(' ')
    for word in wordlist:
        if word not in exclude_words:
            try:
                vocabulary[word] += 1
            except KeyError:
                vocabulary[word] = 1


In [25]:
vocabulary['edge']

70

The above 70 is the number of times dismissals contain the word edge in them. Lets see all these dismissals and see the pattern of words that will allow us to catch outside edge. I think that the best way to make sure that we have all the correct words is to get sets of dismissals from different phrases and then take the intercept of all these dismissals.

Let us no search all dismissals and find the times they contain edge

In [26]:
edge_dismissals = af.search_for_keywords(dismissals_list, keywords=['outside edge', 'drive', 'outside off', 'reach', 'slip'], exclude_words = ['top edge', 'top-edge'], return_matching=True)

In [27]:
edge_dismissals[1]

["That is the end of virat's debut innings, he is gone nibbling at one outside off and baugh has gobbled up the appetiser before lunch, was some distance from virat, not much need to play at it, but for once virat pressed forward, and got so close to the ball that he had to play at it, and ended up getting a healthy edge through to the keeper, a happy bunch of west indians will take lunch,",
 'Rampaul has taken off, his team-mates rush after him, virat has been bounced out, nothing shot, nothing at all actually, just caught on the crease shuffling across, he was probably expecting the bouncer as he was crouching a tad, instead rampaul gets it to shoot off from back of a length and straighten as well, and before virat could do anything, the ball had taken the glove and lobbed high to second slip',
 "Kohli's day just got worse, a loose drive away from his body, it was short of a length and outside off, perhaps should have left it alone, he decides to drive on the up instead, though edwar

Ok, so we manually went through and labelled a number of dismissals, let us load these back in and then we can properly see how many of Kohli's dismissals were actually becuase of driving or cover driving in general.

In [28]:
yes_count = 0
yes_dismissals = []
with open('D:\Documents\Cricket\data_analysis\cricket_data_analysis\data\labelled_data\labelled_drive_dismisals.txt', 'r') as file:
    for line in file.readlines():
        line = line.split('label:')
        label = line[1]
        if label.strip().lower() == 'yes':
            yes_count += 1
            yes_dismissals.append(line[0])

print("Kohli's dismissals that are cover drives:")
print(yes_count)

Kohli's dismissals that are cover drives:
54


Let us now trend these dismissals. I want to know the average score of all these dismissals and I want to know how early in the innings that each of these dismissals happened. For this we are gonna need contributions, as well as knowing what match these drives happened in. From the match function we can get the index of every one of these dismissals and then we can use that and our label ones and zeros to basically figure out which inning the dismisal was in. Or maybe a better way is to get the match id of each of the dismissals. 

Objctive: Get the match id of every one of the cover drive dismissals.

In [29]:
yes_dismissals[0]

'"That is the end of virat\'s debut innings, he is gone nibbling at one outside off and baugh has gobbled up the appetiser before lunch, was some distance from virat, not much need to play at it, but for once virat pressed forward, and got so close to the ball that he had to play at it, and ended up getting a healthy edge through to the keeper, a happy bunch of west indians will take lunch,", '

In [30]:
drive_dismissal_indices = []
j = 0
for i,dismissal in enumerate(dismissals_list):
    print(dismissal.strip('"').strip("'").strip().lower(), yes_dismissals[j].strip('"').strip("'").strip().lower())
    if dismissal.strip('"').strip("'").strip().lower()[:30] == yes_dismissals[j].strip('"').strip("'").strip().lower()[:30]:
        j += 1
        drive_dismissal_indices.append(i)


that is the end of virat's debut innings, he is gone nibbling at one outside off and baugh has gobbled up the appetiser before lunch, was some distance from virat, not much need to play at it, but for once virat pressed forward, and got so close to the ball that he had to play at it, and ended up getting a healthy edge through to the keeper, a happy bunch of west indians will take lunch, that is the end of virat's debut innings, he is gone nibbling at one outside off and baugh has gobbled up the appetiser before lunch, was some distance from virat, not much need to play at it, but for once virat pressed forward, and got so close to the ball that he had to play at it, and ended up getting a healthy edge through to the keeper, a happy bunch of west indians will take lunch,",
soft dismissal! short ball, aimed into the ribs, but headed down the leg side, kohli flirts with it, and west indies' appeal for a catch down the leg side is spontaneous. harper agrees, and a disappointed kohli goes 

In [31]:
KOHLI_ID_COMMS = int(af.get_player_map(match_data.MatchData(kohli_matches[0]), 'player_id', 'object_id')[int(KOHLI_ID)])
drive_dismissal_indices = []
j = 0
for i, _match in enumerate(kohli_comms):
    try:
        dismissal = _match.iloc[-1]
        #dismissal = _match[(_match.isWicket == True) & (_match.batsmanPlayerId == KOHLI_ID_COMMS)]
        dismissal = dismissal.commentTextItems
        #print(dismissal.strip('"').strip("'").strip().lower()[:30])
        #print(yes_dismissals[j].strip('"').strip("'").strip().lower()[:30])
        if dismissal.strip('"').strip("'").strip().lower()[:30] == yes_dismissals[j].strip('"').strip("'").strip().lower()[:30]:
            j += 1
            drive_dismissal_indices.append(i)
    except IndexError:
        pass
        #print(dismissal)



2022-07-29 17:55:12,862 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\match_jsons\489226_match_jsons.json
2022-07-29 17:55:12,866 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\htmls\489226_html.html
2022-07-29 17:55:13,033 - INFO - Match ID 489226: Loading match comms
2022-07-29 17:55:13,034 - INFO - Loading data from d:\Documents\Cricket\data_analysis\cricket_data_analysis\data\full_comms\489226_full_comms.json


In [32]:
drive_dismissal_indices

[0,
 3,
 7,
 9,
 11,
 18,
 20,
 23,
 32,
 33,
 39,
 41,
 43,
 45,
 47,
 48,
 50,
 55,
 58,
 59,
 64,
 65,
 67,
 72,
 78,
 84,
 85,
 92,
 96,
 100,
 110,
 116,
 118,
 120,
 121,
 122,
 124,
 126,
 127,
 133,
 141,
 144,
 146,
 147,
 149,
 156,
 157,
 158,
 159,
 164,
 165,
 166,
 167,
 172]

In [33]:
match_list_by_inning = [inning['match_id'] for inning in kohli_innings]

In [34]:
len(match_list_by_inning)

173

In [35]:
cover_drive_dismissal_match_ids = [match_list_by_inning[i] for i in drive_dismissal_indices]

In [36]:
cover_drive_dismissal_match_ids

['489226',
 '489227',
 '518950',
 '518951',
 '518952',
 '565806',
 '565807',
 '565808',
 '676527',
 '648665',
 '667653',
 '667711',
 '667713',
 '667715',
 '667717',
 '667717',
 '667719',
 '754741',
 '754743',
 '870729',
 '895777',
 '895777',
 '903603',
 '1022593',
 '1030215',
 '1034811',
 '1034811',
 '1062573',
 '1062575',
 '1109604',
 '1122278',
 '1119551',
 '1119552',
 '1119553',
 '1119553',
 '1157752',
 '1144993',
 '1144994',
 '1144994',
 '1188629',
 '1187685',
 '1187686',
 '1223869',
 '1243384',
 '1243385',
 '1239544',
 '1239544',
 '1239545',
 '1239545',
 '1277079',
 '1277079',
 '1277081',
 '1277081',
 '1320741']

Now that we have the cover drive match ids, we can figure out if there is a trend in how early these dismissals are happening, is it the case that Kohli gets out in different ways based on how many balls he has faced, does he get out to cover drives only early in his innings, and finally, how many cover drives does Kohli play in this innings where he gets out to cover drives

In [41]:
cover_drive_match_totals = [kohli_innings[i] for i in drive_dismissal_indices]

In [42]:
cover_drive_match_totals_df = pd.DataFrame(cover_drive_match_totals)
cover_drive_match_totals_df.head()

,inning,runs,balls_faced,fours,six,dot_balls,not_out,how_out,date,team,opposition,ground,continent,match_id
0,1,4,10,1,0,9,False,caught,2011-06-20,6,4,200,Americas,489226
1,3,27,107,1,1,87,False,caught,2011-06-28,6,4,199,Americas,489227
2,2,11,21,1,0,14,False,caught,2011-12-26,6,2,61,Oceania,518950
3,1,23,41,3,0,32,False,caught,2012-01-03,6,2,132,Oceania,518951
4,1,44,82,6,0,61,False,caught,2012-01-13,6,2,213,Oceania,518952


In [43]:
cover_drive_match_totals_df.describe()

,inning,runs,balls_faced,fours,six,dot_balls
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,1.888889,41.018519,80.814815,4.796296,0.055556,58.370370
std,0.984151,46.748823,75.606747,5.360159,0.231212,51.061671
min,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,1.000000,11.000000,24.250000,1.000000,0.000000,18.500000
50%,2.000000,22.000000,62.500000,3.000000,0.000000,46.000000
75%,3.000000,48.250000,106.000000,6.000000,0.000000,75.500000
max,4.000000,200.000000,283.000000,24.000000,1.000000,192.000000


That is very very interesting, Kohli's innings when he gets out from driving look much the same as his normal stats. Nothing drastically different.